In [8]:
# pip install nltk

In [9]:
import nltk
import re
from nltk.stem import PorterStemmer

In [10]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Abdallah.Ayman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abdallah.Ayman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Abdallah.Ayman\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [12]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

In [13]:
print(len(positive_tweets))
print(len(negative_tweets))

5000
5000


In [14]:
def clean_text(tweet):
    #clean text
    tweet=re.sub('(#|@)\w*',"",tweet)# \w [a-z|A_Z|0-9|_] #remoce hashtage ,username
    tweet=re.sub("https?:\/\/\S+","",tweet) #remove hyperlink
    tweet=re.sub("(\?|!)+"," ",tweet) #remve (?!)
    tweet=re.sub("\s\d+\s","",tweet) # 33
    tweet=re.sub("(\.|\,)+","",tweet) #remove . ,
    tweet=re.sub("^\s+","",tweet) #remove space ^ >> start of string
    tweet=re.sub("\s+$","",tweet)#remove space  $ >> at the end of the string
    tweet=re.sub(":","",tweet)
    tweet=re.sub("[_:()\\\]","",tweet)

    return tweet

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\/'
<>:5: SyntaxWarning: invalid escape sequence '\?'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\]'
<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\/'
<>:5: SyntaxWarning: invalid escape sequence '\?'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\]'
C:\Users\Abdallah.Ayman\AppData\Local\Temp\ipykernel_1548\2585229808.py:3: SyntaxWarning: invalid escape sequence '\w'
  tweet=re.sub('(#|@)\w*',"",tweet)# \w [a-z|A_Z|0-9|_] #remoce hashtage ,usern

In [15]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [16]:
def process_sentence(tweets):
  clean_tweets = []
  for tweet in tweets:
    tweet =  clean_text(tweet)
    tweet = nltk.word_tokenize(tweet)
    c_tweets = [word.lower() for word in tweet if word.lower() not in stop_words]
    ps= PorterStemmer()
    clean_tweet = [ps.stem(word) for word in c_tweets]
    clean_tweets.append(clean_tweet)
  return clean_tweets



In [17]:

positive_tweets=process_sentence(positive_tweets)
negative_tweets=process_sentence(negative_tweets)

In [18]:
for i in range(10):
  print(positive_tweets[i])

['top', 'engag', 'member', 'commun', 'week']
['hey', 'jame', 'odd', '/', 'pleas', 'call', 'contact', 'centr', 'onand', 'abl', 'assist', 'mani', 'thank']
['listen', 'last', 'night', 'bleed', 'amaz', 'track', 'scotland']
['congrat']
['yeaaaah', 'yippppi', 'accnt', 'verifi', 'rqst', 'succeed', 'got', 'blue', 'tick', 'mark', 'fb', 'profil', 'inday']
['one', 'irresist']
["n't", 'like', 'keep', 'love', 'custom', 'wait', 'long', 'hope', 'enjoy', 'happi', 'friday', '-', 'lwwf']
['second', 'thought', '’', 'enough', 'time', 'dd', 'new', 'short', 'enter', 'system', 'sheep', 'must', 'buy']
['jgh', 'go', 'bayan', 'bye']
['act', 'mischiev', 'call', 'etl', 'layer', 'in-hous', 'wareh', 'app', 'katamari', 'well…', 'name', 'impli', 'p']


In [19]:
positive_labels= [1]*len(positive_tweets)
negative_labels= [0]*len(negative_tweets)
positive_labels.extend(negative_labels)
positive_tweets.extend(negative_tweets)


In [20]:
labels =  positive_labels
tweets = positive_tweets
print(len(labels))
print(len(tweets))

10000
10000


In [21]:
import random
zip_list = list(zip(tweets,labels))
random.shuffle(zip_list)
tweets,labels = zip(*zip_list)

In [22]:
def build_freq(tweets,labels):
    freq={}

    #iterat tweets
    for i in range(len(tweets)):
        #iterate on each word in each tweets
        for word in tweets[i]:
            key=word
            if key not in freq.keys():### frist time
                if labels[i]==1:
                    freq[key]=[1,0]#positive class
                else:
                    freq[key]=[0,1]#negative class
            else:
                if labels[i]==1:### exist before
                    freq[key][0]+=1
                else:
                    freq[key][1]+=1

    return freq

In [23]:
freq_table = build_freq(tweets,labels)

In [24]:
sum_pos_freq = 0
sum_Neg_freq = 0
for key in freq_table.keys():
  sum_pos_freq += freq_table[key][0]
  sum_Neg_freq += freq_table[key][1]

print(sum_pos_freq)
print(sum_Neg_freq)

30219
29142


In [25]:
V= len(freq_table)
print(V)

10089


In [26]:
def build_probability(freq_table, sum_pos_freq, sum_Neg_freq, V):
  prob_table={}
  for key in freq_table.keys():
    prob_table[key]= [(freq_table[key][0]+1)/(sum_pos_freq+V), (freq_table[key][1]+1)/(sum_Neg_freq+V)]
  return prob_table


In [27]:
prob_dict = build_probability(freq_table=freq_table, sum_pos_freq=sum_pos_freq, sum_Neg_freq=sum_Neg_freq, V=V)

In [28]:
import numpy as np
def Naive_Bayes_inference(tweets, prop_dict):
  results = []
  for tweet in tweets:
    result = 0
    for word in tweet:
      try:
        result += np.log(prob_dict[word][0]/ prob_dict[word][1])# ratio
      except:
        result +=0
    results.append(result)
  return results

In [29]:
y_pred = Naive_Bayes_inference(tweets, prop_dict=freq_table)

In [30]:
y_p = [i if y>=0 else 0 for y  in y_pred]

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(labels, y_p)

0.4374

In [32]:
test_tweets=["@metalgear_jp @Kojima_Hideo I want you're T-shirts ! They are so cool ! :D","Stats for the day have arrived. 2 new followers and NO unfollowers :) via http://t.co/xxlXs6xYwe.",
             "Dang that is some rad @AbzuGame #fanart! :D https://t.co/bI8k8tb9ht","Can u feel it? :((:( #exo http://t.co/ghsa262ORm","@seanactual You mean you're not offering? :("]

In [33]:
clean_tweet = process_sentence(test_tweets)
clean_tweet

[['want', "'re", 't-shirt', 'cool'],
 ['stat', 'day', 'arrivednew', 'follow', 'unfollow', 'via'],
 ['dang', 'rad'],
 ['u', 'feel'],
 ['mean', "'re", 'offer']]

In [34]:
Naive_Bayes_inference(clean_tweet, prob_dict)

[np.float64(2.7126399671171493),
 np.float64(13.008719656930369),
 np.float64(1.0444468690167477),
 np.float64(-1.0111916407347108),
 np.float64(-0.22540661467585196)]